objetivo de crossvalidation es poder entrenar y testear con toda la base
validacion: evaluar como le iria a mi algoritmos en datos que nunca vio, se busca encontrar los mejores hiperparametros
estrategias
separacion entre train y test


2 objetivos , encontrar hiper, como nos va a ir cuando lo llevemos a la practica

In [ ]:
#traintestsplit separa en train y test 80 y 20

In [ ]:
#K fold hace lo mismo K veces

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, KFold, ParameterGrid
from lightgbm import LGBMClassifier
import xgboost as xgb

In [ ]:
train = pd.read_csv("/kaggle/input/petfinder-adoption-prediction/train/train.csv", index_col = "PetID")

In [ ]:
#KFold?

In [ ]:
train.drop('Name',axis=1, inplace = True)
train.drop('RescuerID',axis=1, inplace = True)
train.drop('Description',axis=1, inplace = True)


In [ ]:
#con test simulo los datos del futuro, como si fueran de produccion
train, test = train_test_split(train, test_size=0.2, random_state = 1)
folds = [train.index[idx] for idx, _ in KFold(n_splits = 4, random_state = 1 , shuffle = True).split(train)]

In [ ]:
#train.columns

In [ ]:
#folds

In [ ]:
#el 80 por ciento restante lo separo en pedacitos de 20
####folds = [Kfold(n_splits = 4, random_state = 1, shuffle=True).split(train)]
# entreno en 3 pedacitos y hago validaciones en el 4
# despues entreno en otros 3 y valido con el que queda y asi
#
# elegimos hiperparametros y una vez que lo elegi estimo como me va a ir en la practica con testeo

In [ ]:
#empiecen iterando sobre 2 algoritmos con 2 hiperparametros

In [ ]:
#parameterGrid
#params = {'max-depth':[1,100], "otro":[1,2,3]}
#list(ParameterGrid)#devuelve una lista de diccionarios con todas las posibles combinaciones
#for depths in max-depth:
#    for otro in otros:
#        Algo(depth=depth, otro= otro)
#problema1 agrego parametro debo modificar todo
#problema2 el mismo codigo que funciona para un algoritmo no funciona para otro

In [ ]:
#** toma un diccionario y lo convierte en 
#def f(a,b,c):
#    print(f'{a},{b},{c}')
#    f(a=1,b=2,c=3) = f(**params)

In [ ]:
valid_preds = pd.DataFrame(index=train.index)
valid_preds

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

candidates = [
    {"model": LogisticRegression, "params": {"penalty": ["l2"], "C": [0.1, 1, 10]}},
    {"model": LogisticRegression, "params": {"penalty": ["l1"], "C": [0.1, 1, 10], "solver": ["liblinear"]}},
    {"model": RandomForestClassifier, "params": {"max_depth": [None, 100], "min_samples_split": [40, 30]}},
    {"model": RandomForestClassifier, "params": {"max_depth": [10, 25], "min_samples_split": [2, 5]}}
]
valid_preds = pd.DataFrame(index=train.index)
test_preds = pd.DataFrame(index=test.index)
for candidate in candidates:
    for params in ParameterGrid(candidate["params"]):
        name = candidate["model"].__name__ + ";" + ";".join([f"{k}_{v}" for k, v in params.items()])
        fold_preds = []
        fold_test_preds = []
        for train_idx in folds:
            X_train = train.loc[train_idx].drop("AdoptionSpeed", axis=1)
            y_train = train.loc[train_idx, "AdoptionSpeed"]
            
            X_valid = train.drop(train_idx).drop("AdoptionSpeed", axis=1)
            y_valid = train.loc[X_valid.index, "AdoptionSpeed"]
            
            model = candidate["model"](**params)
            model.fit(X_train, y_train)
            fold_preds.append(pd.Series(model.predict(X_valid), index=X_valid.index))#concat
            fold_test_preds.append(pd.Series(model.predict(test.drop("AdoptionSpeed", axis = 1)),index= test.index))#
        fold_preds = pd.concat(fold_preds)
        fold_test_preds = pd.concat(fold_test_preds,axis = 1)
        break
    break


In [ ]:
name

In [ ]:
fold_test_preds

In [ ]:
a = fold_test_preds.mode(axis = 1)[0].rename(columns = {'0':"name"')

In [ ]:
a